#Sistem Rekomendasi dengan DNN dan SVM pada Last.FM Dataset

## Import Library

In [ ]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import codecs

## Melihat Dataset

In [ ]:
user_artists_df = pd.read_table('user_artists.dat', encoding="utf-8",sep="\t")
user_artists_df

FileNotFoundError: ignored

In [ ]:
artists_df = pd.read_table('artists.dat',encoding="utf-8",sep="\t")
artists_df

,id,name,url,pictureURL
0,1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
1,2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
2,3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
3,4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
4,5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...
...,...,...,...,...
17627,18741,Diamanda Galás,http://www.last.fm/music/Diamanda+Gal%C3%A1s,http://userserve-ak.last.fm/serve/252/16352971...
17628,18742,Aya RL,http://www.last.fm/music/Aya+RL,http://userserve-ak.last.fm/serve/252/207445.jpg
17629,18743,Coptic Rain,http://www.last.fm/music/Coptic+Rain,http://userserve-ak.last.fm/serve/252/344868.jpg
17630,18744,Oz Alchemist,http://www.last.fm/music/Oz+Alchemist,http://userserve-ak.last.fm/serve/252/29297695...


In [ ]:
user_friends_df = pd.read_table('user_friends.dat',encoding="utf-8",sep="\t")
user_friends_df

,userID,friendID
0,2,275
1,2,428
2,2,515
3,2,761
4,2,831
...,...,...
25429,2099,1801
25430,2099,2006
25431,2099,2016
25432,2100,586


In [ ]:
UTT_df = pd.read_table('user_taggedartists-timestamps.dat',encoding="utf-8",sep="\t")
UTT_df

,userID,artistID,tagID,timestamp
0,2,52,13,1238536800000
1,2,52,15,1238536800000
2,2,52,18,1238536800000
3,2,52,21,1238536800000
4,2,52,41,1238536800000
...,...,...,...,...
186474,2100,16437,4,1277935200000
186475,2100,16437,292,1272664800000
186476,2100,16437,2087,1277935200000
186477,2100,16437,2801,1272664800000


In [ ]:
UT_df = pd.read_table('user_taggedartists.dat',encoding="utf-8",sep="\t")
UT_df

,userID,artistID,tagID,day,month,year
0,2,52,13,1,4,2009
1,2,52,15,1,4,2009
2,2,52,18,1,4,2009
3,2,52,21,1,4,2009
4,2,52,41,1,4,2009
...,...,...,...,...,...,...
186474,2100,16437,4,1,7,2010
186475,2100,16437,292,1,5,2010
186476,2100,16437,2087,1,7,2010
186477,2100,16437,2801,1,5,2010


**bold text**## Eksploratory Data Analysis

### Siapa Artis Yang Paling Terkenal?

In [ ]:
user_artists_df = pd.read_table('user_artists.dat', encoding="utf-8",sep="\t")
artists_df = pd.read_table('artists.dat',encoding="utf-8",sep="\t")
ua_merge = user_artists_df.merge(artists_df, left_on="artistID", right_on="id")

NameError: ignored

In [ ]:
ua_merge.tail()

NameError: ignored

In [ ]:
g_weightbyartist = ua_merge.weight.groupby(ua_merge['artistID'])

In [ ]:
# Mengelompokkan jumlah weight berdasarkan nama artist
g_weightbyartist = ua_merge.weight.groupby(ua_merge['artistID'])
# Menghitung jumlah weight pada objek groupby
# Gunakan fungsi -apply & -combine
s_weightbyartist = g_weightbyartist.sum()
# 10 Artist dengan dengan jumlah Weight terbanyak
s1 = s_weightbyartist.sort_values(ascending=False).head(10)
# Melakukan iterasi
for (key,value) in s1.iteritems():
    print('    ',artists_df[artists_df.id==key].name.item(),'(',key,')  ',
          value,sep='')

    Britney Spears(289)  2393140
    Depeche Mode(72)  1301308
    Lady Gaga(89)  1291387
    Christina Aguilera(292)  1058405
    Paramore(498)  963449
    Madonna(67)  921198
    Rihanna(288)  905423
    Shakira(701)  688529
    The Beatles(227)  662116
    Katy Perry(300)  532545


### Siapa Artis Dengan Pendengar Terbanyak?

In [ ]:
# Mengelompokkan jumlah pendengar berdasarkan nama artist
g_listenerbyartist = ua_merge.userID.groupby(ua_merge['artistID'])
# Menghitung jumlah pendengar pada objek groupby
s_listenerbyartist = g_listenerbyartist.count()
# 10 Artist dengan dengan jumlah pendengar terbanyak
s2 = s_listenerbyartist.sort_values(ascending=False).head(10)
for (key,value) in s2.iteritems():
    print('    ',artists_df[artists_df.id==key].name.item(),'(',key,')  ',
          value, sep='')

    Lady Gaga(89)  611
    Britney Spears(289)  522
    Rihanna(288)  484
    The Beatles(227)  480
    Katy Perry(300)  473
    Madonna(67)  429
    Avril Lavigne(333)  417
    Christina Aguilera(292)  407
    Muse(190)  400
    Paramore(498)  399


### Siapa jumlah pengguna Artis tertinggi berdasarkan jumlah pemutaran lagu?

In [ ]:
# Mengelompokkan pengguna artis berdasarkan jumlah
g_weightbyuser = ua_merge.weight.groupby(ua_merge['userID'])
# Menghitung jumlah pengguna artis pada objek groupby
s_weightbyuser = g_weightbyuser.sum()
# 10 pengguna artist dengan dengan jumlah pemutaran lagu terbanyak
s3 =s_weightbyuser.sort_values(ascending=False).head(10)
for (key,value) in s3.iteritems():
    print('    ',key,'  ',value)

     757    480039
     2000    468409
     1418    416349
     1642    388251
     1094    379125
     1942    348527
     2071    338400
     2031    329980
     514    329782
     387    322661


### Siapa artist yang memiliki jumlah pemutaran rata-rata tertinggi per pendengar?

In [ ]:
#Rata-rata pemutaran per pendegar untuk seiap artist
s_avgplaysperlistener = s_weightbyartist/s_listenerbyartist
#Mengurutkan dan menyimpan 10 nilai terbesar
s4 = s_avgplaysperlistener.sort_values(ascending=False).head(10)
#Menampilkan hasil dengan iterasi 
for (key,value) in s4.iteritems():
    print('    ',artists_df[artists_df.id==key].name.item(),'(',key,')  ',sep='')
    print('        Jumlah Total Pemutaran Lagu: ', s_weightbyartist[key])
    print('        Jumlah Total Pendengar: ', s_listenerbyartist[key])
    print('        Rata-rata: ', "{0:.2f}".format(value))

    Viking Quest(8388)  
        Jumlah Total Pemutaran Lagu:  35323
        Jumlah Total Pendengar:  1
        Rata-rata:  35323.00
    Tyler Adam(6373)  
        Jumlah Total Pemutaran Lagu:  30614
        Jumlah Total Pendengar:  1
        Rata-rata:  30614.00
    Rytmus(18121)  
        Jumlah Total Pemutaran Lagu:  23462
        Jumlah Total Pendengar:  1
        Rata-rata:  23462.00
    Johnny Hallyday(8308)  
        Jumlah Total Pemutaran Lagu:  32995
        Jumlah Total Pendengar:  2
        Rata-rata:  16497.50
    Dicky Dixon(14986)  
        Jumlah Total Pemutaran Lagu:  15345
        Jumlah Total Pendengar:  1
        Rata-rata:  15345.00
    RICHARD DIXON-COMPOSER(14987)  
        Jumlah Total Pemutaran Lagu:  14082
        Jumlah Total Pendengar:  1
        Rata-rata:  14082.00
    Thalía(792)  
        Jumlah Total Pemutaran Lagu:  350035
        Jumlah Total Pendengar:  26
        Rata-rata:  13462.88
    80kidz(15075)  
        Jumlah Total Pemutaran Lagu:  12520
   

### Artist dengan setidaknya 50 pendengar yang memiliki rata-rata jumlah pemutaran per pendengar tertinggi?

In [ ]:
# buat dataframe baru untuk menyimpan artis, pendengar, dan permainan rata-rata per informasi pendengar
df5 = pd.DataFrame({'listener/artist':s_listenerbyartist,
                    'avgplays/listener':s_avgplaysperlistener, 
                    'artistID': s_listenerbyartist.index})
# filter artis dengan lebih dari 50 pendengar terlebih dahulu dan mengambil 10 artis teratas dengan rata-rata pemutaran per pendengar
s5 = df5[df5['listener/artist']>50]['avgplays/listener'].sort_values(ascending=False).head(10)
for (key,value) in s5.iteritems():
    print('    ',artists_df[artists_df.id==key].name.item(),'(',key,')  ',sep='')
    print('        Jumlah Total Pemutaran Lagu: ', s_weightbyartist[key])
    print('        Jumlah Total Pendengar: ', s_listenerbyartist[key])
    print('        Rata-rata: ',"{0:.2f}".format(value))

    Depeche Mode(72)  
        Jumlah Total Pemutaran Lagu:  1301308
        Jumlah Total Pendengar:  282
        Rata-rata:  4614.57
    Britney Spears(289)  
        Jumlah Total Pemutaran Lagu:  2393140
        Jumlah Total Pendengar:  522
        Rata-rata:  4584.56
    In Flames(503)  
        Jumlah Total Pemutaran Lagu:  237148
        Jumlah Total Pendengar:  67
        Rata-rata:  3539.52
    Duran Duran(51)  
        Jumlah Total Pemutaran Lagu:  348919
        Jumlah Total Pendengar:  111
        Rata-rata:  3143.41
    All Time Low(687)  
        Jumlah Total Pemutaran Lagu:  215777
        Jumlah Total Pendengar:  77
        Rata-rata:  2802.30
    Blur(203)  
        Jumlah Total Pemutaran Lagu:  318221
        Jumlah Total Pendengar:  114
        Rata-rata:  2791.41
    U2(511)  
        Jumlah Total Pemutaran Lagu:  493024
        Jumlah Total Pendengar:  185
        Rata-rata:  2664.99
    Christina Aguilera(292)  
        Jumlah Total Pemutaran Lagu:  1058405
        

### Apakah pengguna dengan lima teman atau lebih mendengarkan lebih banyak lagu?

In [ ]:
# memuat data teman pengguna ke dalam dataframe
user_friends_df = pd.read_table('user_friends.dat',encoding="utf-8",sep="\t")
# hitung jumlah teman setiap penggunat
s6_uf = user_friends_df.friendID.groupby(user_friends_df.userID).nunique()
# hitung jumlah permainan setiap pengguna
s6_uw = user_artists_df.weight.groupby(user_artists_df.userID).sum()
# buat teman penyimpanan bingkai data baru, mainkan informasi untuk setiap pengguna
df6 = pd.DataFrame({'friend':s6_uf, 'weight':s6_uw, 'userID': s6_uf.index})
# hitung jumlah permainan rata-rata oleh pengguna yang memiliki kurang dari 5 teman
ls_friend = (df6[df6.friend<5]['weight'].sum())/(df6[df6.friend<5]['userID'].count())
# hitung jumlah permainan rata-rata oleh pengguna dengan> = 5 teman
mr_friend = (df6[df6.friend>=5]['weight'].sum())/(df6[df6.friend>=5]['userID'].count())
print('    Jumlah lagu yang didengarkan pengguna (<5 teman): ', "{0:.2f}".format(ls_friend))
print('    Jumlah lagu yang didengarkan pengguna (> = 5 teman): ', "{0:.2f}".format(mr_friend))
if ls_friend < mr_friend:
    print('    Jadi pengguna dengan lima atau lebih teman mendengarkan lebih banyak lagu')
elif ls_friend > mr_friend:
    print('    Jadi pengguna dengan lima atau lebih teman mendengarkan lebih sedikit lagu')
else:
    print('    Jumlah teman tidak mempengaruhi.')

    Jumlah lagu yang didengarkan pengguna (<5 teman):  24513.86
    Jumlah lagu yang didengarkan pengguna (> = 5 teman):  44676.33
    Jadi pengguna dengan lima atau lebih teman mendengarkan lebih banyak lagu


### Seberapa miripkah sepasang artist?

*   List item
*   List item



In [ ]:
# mendefinisikan fungsi untuk menghitung kemiripan antara dua artis menggunakan indeks Jaccard
def artist_sim(aid1, aid2):
    # membuat himpunan pasangan artist
    artist_1 = user_artists_df[user_artists_df['artistID']==aid1]['userID']
    artist_2 = user_artists_df[user_artists_df['artistID']==aid2]['userID']
    # dapatkan intereksi yang ditetapkan pengguna sebagai pembilang indeks Jaccard
    artist_intersection = pd.Series(list(set(artist_1).intersection(set(artist_2))))
    # dapatkan serikat yang ditetapkan pengguna sebagai nonminator indeks Jaccard
    artist_union = pd.Series(list(set(artist_1).union(set(artist_2))))
    # menghitung Jaccard index
    jaccard_index =  artist_intersection.count()/artist_union.count()
    return jaccard_index
s_artistpair = Series([562,89,289,289,67,735], index=[735,735,735,89,89,67]) 
print('    (Di bawah ini adalah contoh tingkat kesamaan dua orang artist:)')
for (key,value) in s_artistpair.iteritems():
    print('   ',artists_df[artists_df.id==key].name.item(),' ',
          artists_df[artists_df.id==value].name.item(),'      -Jaccard index: '
          ,"{0:.2f}".format(artist_sim(key, value)))

    (Di bawah ini adalah contoh tingkat kesamaan dua orang artist:)
    The Rolling Stones   The Who       -Jaccard index:  0.25
    The Rolling Stones   Lady Gaga       -Jaccard index:  0.02
    The Rolling Stones   Britney Spears       -Jaccard index:  0.01
    Lady Gaga   Britney Spears       -Jaccard index:  0.63
    Lady Gaga   Madonna       -Jaccard index:  0.42
    Madonna   The Rolling Stones       -Jaccard index:  0.03


### Analisis artis dengan tag terbanyak

In [ ]:
tag_artist = pd.read_table('user_taggedartists.dat', encoding="utf-8",sep="\t")
# dapatkan 10 artis dengan tag teratas saat ini
toptag_artist = tag_artist .tagID.groupby(tag_artist.artistID).count().sort_values(ascending=False).head(10)
# gabungkan kolom tahun dan bulan
tag_artist.year = tag_artist.year.map(str) + '-' + tag_artist.month.map(str)
tag_artist  = tag_artist.rename(columns={'year':'year-month'})
# gunakan indeks hierarki untuk menghitung nomor tag artis per bulan setiap tahun
tag = tag_artist.groupby(['year-month','artistID']).count()
tag = tag.rename(columns={'tagID':'tagCounts'})
# hapus catatan abnormal dari abad terakhir
tag = tag[4:]
# pengurutan multi-indeks berdasarkan jumlah tag
tag = tag.sort_values(by='tagCounts', ascending = False).swaplevel(0,1).sort_index(level=1, sort_remaining = False).swaplevel(0,1)
# pertahankan 10 teratas untuk setiap bulanh
tag = tag.groupby(level='year-month').head(10)
# reset indeks untuk tag dataframe, ubah id artis ke kolomn
tag.reset_index(inplace=True)
# gunakan for loop untuk mengulang setiap artis di 10 teratas saat ini
# periksa apakah artist ada dalam history 10 teratas untuk setiap bulan, jika demikian, terus iterasikan
# cetak saat pertama kali  masuk 10 besar 
for artist, weight in toptag_artist.iteritems():
   i = 0
   print('    ',artists_df[artists_df.id==artist].name.item(),'(',artist,'):  num tags = ',toptag_artist[artist],sep='')
   for index, row in tag.iterrows():
       if artist == row['artistID']:
           i += 1
           if i ==1:
               print("      pertama kali masuk 10 besar:", row['year-month'])
   print("      total bulan di 10 teratas: ", i,'\n')

    Britney Spears(289):  num tags = 931
      pertama kali masuk 10 besar: 2006-9
      total bulan di 10 teratas:  26 

    Lady Gaga(89):  num tags = 767
      pertama kali masuk 10 besar: 2008-12
      total bulan di 10 teratas:  18 

    Christina Aguilera(292):  num tags = 762
      pertama kali masuk 10 besar: 2006-8
      total bulan di 10 teratas:  15 

    Madonna(67):  num tags = 730
      pertama kali masuk 10 besar: 2006-8
      total bulan di 10 teratas:  15 

    Depeche Mode(72):  num tags = 701
      pertama kali masuk 10 besar: 2005-12
      total bulan di 10 teratas:  19 

    The Beatles(227):  num tags = 598
      pertama kali masuk 10 besar: 2005-11
      total bulan di 10 teratas:  15 

    Muse(190):  num tags = 595
      pertama kali masuk 10 besar: 2006-11
      total bulan di 10 teratas:  11 

    Radiohead(154):  num tags = 563
      pertama kali masuk 10 besar: 2005-10
      total bulan di 10 teratas:  14 

    Rihanna(288):  num tags = 528
      pertama ka

## Recomender System Using Deep Neural Networks

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
weight =user_artists_df
weight.tail(25)

,userID,artistID,weight,user,artist,weight_app
92809,2100,8320,284.0,1891,8127,10.161372
92810,2100,8322,650.0,1891,8129,10.859745
92811,2100,8323,456.0,1891,8130,6.657830
92812,2100,8324,1068.0,1891,8131,19.528944
92813,2100,8326,626.0,1891,8133,9.971657
92814,2100,8327,613.0,1891,8134,56.541126
92815,2100,8332,655.0,1891,8139,15.406197
92816,2100,8344,640.0,1891,8151,18.520147
92817,2100,8525,232.0,1891,8324,8.852034
92818,2100,8529,429.0,1891,8328,2.562800


In [ ]:
artist = artists_df[['id', 'name']]
artist.tail(25)

,id,name
17607,18718,Onra & Quetzal
17608,18719,Controller 7
17609,18720,"FORCE OF NATURE, Nujabes, fat jon"
17610,18721,padmo'
17611,18722,Fingathing
17612,18723,Electrosoul System
17613,18724,Nostalgia 77
17614,18725,Mortifera
17615,18726,Nyktalgia
17616,18727,Atsakau niekadA


In [ ]:
# Membuat Matriks User & Item
g = weight.groupby('userID')['weight'].count()
top_users = g.sort_values(ascending=False)[:15]
g = weight.groupby('artistID')['weight'].count()
top_movies = g.sort_values(ascending=False)[:15]
top_r = weight.join(top_users, rsuffix='_r', how='inner', on='userID')
top_r = top_r.join(top_movies, rsuffix='_r', how='inner', on='artistID')
pd.crosstab(top_r.userID, top_r.userID, top_r.weight, aggfunc=np.sum)

userID,677,678,679,680,681,682,684,685,686,688,689,690
userID,,,,,,,,,,,,
677,1086.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
678,NaN,332.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
679,NaN,NaN,8209.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
680,NaN,NaN,NaN,13330.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
681,NaN,NaN,NaN,NaN,465.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
682,NaN,NaN,NaN,NaN,NaN,4169.0,NaN,NaN,NaN,NaN,NaN,NaN
684,NaN,NaN,NaN,NaN,NaN,NaN,124191.0,NaN,NaN,NaN,NaN,NaN
685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6452.0,NaN,NaN,NaN,NaN
686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10124.0,NaN,NaN,NaN


In [ ]:
#Menyiapkan parameter
user_enc = LabelEncoder()
weight['user'] = user_enc.fit_transform(weight['userID'].values)
n_users = weight['user'].nunique()
item_enc = LabelEncoder()
weight['artist'] = item_enc.fit_transform(weight['artistID'].values)
n_artists = weight['artist'].nunique()
weight['weight'] = weight['weight'].values.astype(np.float32)
min_weight = min(weight['weight'])
max_weight = max(weight['weight'])
n_users, n_artists, min_weight, max_weight

(1892, 17632, 1.0, 352698.0)

In [ ]:
X = weight[['user', 'artist']].values
y = weight['weight'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((83550, 2), (9284, 2), (83550,), (9284,))

In [ ]:
n_factors = 50
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

In [ ]:
from keras.models import Model
from keras.layers import Input, Reshape, Dot, Concatenate, Dropout, Dense
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.regularizers import l2

from keras.layers import Add, Activation, Lambda
class EmbeddingLayer:
    def __init__(self, n_items, n_factors):
        self.n_items = n_items
        self.n_factors = n_factors
    
    def __call__(self, x):
        x = Embedding(self.n_items, self.n_factors, embeddings_initializer='he_normal',
                      embeddings_regularizer=l2(1e-6))(x)
        x = Reshape((self.n_factors,))(x)
        return x

In [ ]:
def RecommenderV1(n_users, n_artists, n_factors):
    user = Input(shape=(1,))
    u = Embedding(n_users, n_factors, embeddings_initializer='he_normal',
                  embeddings_regularizer=l2(1e-6))(user)
    u = Reshape((n_factors,))(u)
    
    artist = Input(shape=(1,))
    m = Embedding(n_artists, n_factors, embeddings_initializer='he_normal',
                  embeddings_regularizer=l2(1e-6))(artist)
    m = Reshape((n_factors,))(m)
    
    x = Dot(axes=1)([u, m])
    model = Model(inputs=[user, artist], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

In [ ]:
model = RecommenderV1(n_users, n_artists, n_factors)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_20 (Embedding)        (None, 1, 50)        94600       input_23[0][0]                   
__________________________________________________________________________________________________
embedding_21 (Embedding)        (None, 1, 50)        881600      input_24[0][0]                   
____________________________________________________________________________________________

In [ ]:
history = model.fit(x=X_train_array, y=y_train, batch_size=64, epochs=5,
                    verbose=1, validation_data=(X_test_array, y_test))

Epoch 1/5
1306/1306 [==============================] - 11s 8ms/step - loss: 12724701.9208 - val_loss: 7971597.0000
Epoch 2/5
1306/1306 [==============================] - 11s 9ms/step - loss: 13217635.6793 - val_loss: 7966999.0000
Epoch 3/5
1306/1306 [==============================] - 11s 8ms/step - loss: 14852879.3656 - val_loss: 7958777.5000
Epoch 4/5
1306/1306 [==============================] - 11s 8ms/step - loss: 17755640.5038 - val_loss: 7947832.5000
Epoch 5/5
1306/1306 [==============================] - 11s 8ms/step - loss: 18459205.2243 - val_loss: 7934636.0000


In [ ]:
def RecommenderV2(n_users, n_artists, n_factors, min_weight, max_weight):
    user = Input(shape=(1,))
    u = EmbeddingLayer(n_users, n_factors)(user)
    ub = EmbeddingLayer(n_users, 1)(user)
    
    artist = Input(shape=(1,))
    m = EmbeddingLayer(n_artists, n_factors)(artist)
    mb = EmbeddingLayer(n_artists, 1)(artist)
    x = Dot(axes=1)([u, m])
    x = Add()([x, ub, mb])
    x = Activation('sigmoid')(x)
    x = Lambda(lambda x: x * (max_weight - min_weight) + min_weight)(x)
    model = Model(inputs=[user, artist], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

In [ ]:
model = RecommenderV2(n_users, n_artists, n_factors, min_weight, max_weight)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_28 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_26 (Embedding)        (None, 1, 50)        94600       input_27[0][0]                   
__________________________________________________________________________________________________
embedding_28 (Embedding)        (None, 1, 50)        881600      input_28[0][0]                   
____________________________________________________________________________________________

In [ ]:
history = model.fit(x=X_train_array, y=y_train, batch_size=64, epochs=5,
                    verbose=1, validation_data=(X_test_array, y_test))

Epoch 1/5
1306/1306 [==============================] - 14s 10ms/step - loss: 27105402861.1966 - val_loss: 11141166080.0000
Epoch 2/5
1306/1306 [==============================] - 13s 10ms/step - loss: 7682393688.9243 - val_loss: 4333473280.0000
Epoch 3/5
1306/1306 [==============================] - 13s 10ms/step - loss: 2064718821.1660 - val_loss: 2939144704.0000
Epoch 4/5
1306/1306 [==============================] - 13s 10ms/step - loss: 615707070.7758 - val_loss: 2491030784.0000
Epoch 5/5
1306/1306 [==============================] - 14s 10ms/step - loss: 208771857.3588 - val_loss: 2305663488.0000


In [ ]:
def RecommenderNet(n_users, n_artists, n_factors, min_weight, max_weight):
    user = Input(shape=(1,))
    u = EmbeddingLayer(n_users, n_factors)(user)
    
    artist = Input(shape=(1,))
    m = EmbeddingLayer(n_artists, n_factors)(artist)
    
    x = Concatenate()([u, m])
    x = Dropout(0.05)(x)
    
    x = Dense(10, kernel_initializer='he_normal')(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    
    x = Dense(1, kernel_initializer='he_normal')(x)
    x = Activation('sigmoid')(x)
    x = Lambda(lambda x: x * (max_weight - min_weight) + min_weight)(x)
    model = Model(inputs=[user, artist], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

In [ ]:
history = model.fit(x=X_train_array, y=y_train, batch_size=64, epochs=5,
                    verbose=1, validation_data=(X_test_array, y_test))

In [ ]:
model = RecommenderNet(n_users, n_artists, n_factors, min_weight, max_weight)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_15 (Embedding)        (None, 1, 50)        94600       input_17[0][0]                   
__________________________________________________________________________________________________
embedding_16 (Embedding)        (None, 1, 50)        881600      input_18[0][0]                   
______________________________________________________________________________________________

In [ ]:
history = model.fit(x=X_train_array, y=y_train, batch_size=64, epochs=5,
                    verbose=1, validation_data=(X_test_array, y_test))

Epoch 1/5
1306/1306 [==============================] - 13s 9ms/step - loss: 10840103034.6136 - val_loss: 10796421.0000
Epoch 2/5
1306/1306 [==============================] - 12s 9ms/step - loss: 1169465119.4369 - val_loss: 9010496.0000
Epoch 3/5
1306/1306 [==============================] - 12s 9ms/step - loss: 696299038.7024 - val_loss: 8198710.5000
Epoch 4/5
1306/1306 [==============================] - 12s 9ms/step - loss: 370660556.5356 - val_loss: 7999912.5000
Epoch 5/5
1306/1306 [==============================] - 12s 9ms/step - loss: 178753667.0849 - val_loss: 7976571.0000


In [ ]:
n_factors = 50
X_pake = [X[:, 0], X[:, 1]]
hasil=model(X_pake)

In [ ]:
weight['weight_app']=np.array(hasil)
weight_new=weight[['userID', 'artistID', 'weight_app']]
weight_new

,userID,artistID,weight_app
0,2,51,2.700617
1,2,52,4.201870
2,2,53,2.383490
3,2,54,2.652724
4,2,55,1.320142
...,...,...,...
92829,2100,18726,3.705580
92830,2100,18727,56.225792
92831,2100,18728,2.212849
92832,2100,18729,3.750654


In [ ]:
# Membuat Matriks User & Item
g = weight.groupby('userID')['weight_app'].count()
top_users = g.sort_values(ascending=False)[:15]
g = weight.groupby('artistID')['weight_app'].count()
top_movies = g.sort_values(ascending=False)[:15]
top_r = weight.join(top_users, rsuffix='_r', how='inner', on='userID')
top_r = top_r.join(top_movies, rsuffix='_r', how='inner', on='artistID')
matriks_user_item=pd.crosstab(top_r.userID, top_r.userID, top_r.weight_app, aggfunc=np.sum)
matriks_user_item

userID,677,678,679,680,681,682,684,685,686,688,689,690
userID,,,,,,,,,,,,
677,7.388812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
678,NaN,8.327539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
679,NaN,NaN,33.898041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
680,NaN,NaN,NaN,22.996532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
681,NaN,NaN,NaN,NaN,2.842769,NaN,NaN,NaN,NaN,NaN,NaN,NaN
682,NaN,NaN,NaN,NaN,NaN,22.500854,NaN,NaN,NaN,NaN,NaN,NaN
684,NaN,NaN,NaN,NaN,NaN,NaN,119.208183,NaN,NaN,NaN,NaN,NaN
685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.985792,NaN,NaN,NaN,NaN
686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.101406,NaN,NaN,NaN


In [ ]:
artists = artists_df
artists

,id,name,url,pictureURL
0,1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
1,2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
2,3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
3,4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
4,5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...
...,...,...,...,...
17627,18741,Diamanda Galás,http://www.last.fm/music/Diamanda+Gal%C3%A1s,http://userserve-ak.last.fm/serve/252/16352971...
17628,18742,Aya RL,http://www.last.fm/music/Aya+RL,http://userserve-ak.last.fm/serve/252/207445.jpg
17629,18743,Coptic Rain,http://www.last.fm/music/Coptic+Rain,http://userserve-ak.last.fm/serve/252/344868.jpg
17630,18744,Oz Alchemist,http://www.last.fm/music/Oz+Alchemist,http://userserve-ak.last.fm/serve/252/29297695...


In [ ]:
weights = weight_new.merge(artists, left_on="artistID", right_on="id")
weights = weights.drop(['url','pictureURL', 'id'],axis=1)

print(weights.shape)
weights.head()

(92834, 4)


,userID,artistID,weight_app,name
0,2,51,2.700617,Duran Duran
1,4,51,11.368856,Duran Duran
2,27,51,5.496796,Duran Duran
3,28,51,7.074401,Duran Duran
4,62,51,8.146610,Duran Duran


In [ ]:
userWeights = weights.pivot_table(index=['userID'],columns=['name'],values='weight_app')
userWeights.head()
print("Before: ",userWeights.shape)
userWeights = userWeights.dropna(thresh=10, axis=1).fillna(0,axis=1)
print("After: ",userWeights.shape)

Before:  (1892, 17632)
After:  (1892, 1530)


In [ ]:
corrMatrix = userWeights.corr(method='pearson')
corrMatrix.head(100)

name,*NSYNC,+44,...And You Will Know Us by the Trail of Dead,12 Stones,2NE1,2Pac,3 Doors Down,30 Seconds to Mars,311,36 Crazyfists,3OH!3,4minute,50 Cent,65daysofstatic,:wumpscut:,A Day to Remember,A Fine Frenzy,A Flock of Seagulls,A Perfect Circle,A Rocket to the Moon,A Silver Mt. Zion,A Skylit Drive,A Tribe Called Quest,A*Teens,ABBA,ABC,AC/DC,AFI,AFX,ASIAN KUNG-FU GENERATION,ATB,Aaliyah,Aaron Carter,Accept,Ace of Base,Adam Lambert,Adam and the Ants,Adele,Adema,Adriana Calcanhotto,...,Yngwie Malmsteen,Yo La Tengo,You Me At Six,ZZ Top,Zebrahead,Zeca Baleiro,Zero 7,Zoé,Zемфира,[unknown],a-ha,blessthefall,blink-182,deadmau5,dredg,happysad,iamamiwhoami,mind.in.a.box,múm,nevershoutnever!,of Montreal,t.A.T.u.,the GazettE,Édith Piaf,Ólafur Arnalds,Агата Кристи,Ария,Дельфин,Кино,Ляпис Трубецкой,Психея,Сплин,倖田來未,宇多田ヒカル,安室奈美恵,植松伸夫,浜崎あゆみ,동방신기,소녀시대,티아라
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
*NSYNC,1.000000,-0.009446,-0.007940,0.000842,-0.011509,-0.012572,0.021569,-0.009598,-0.007575,-0.010259,0.069252,0.032779,0.021656,-0.013165,-0.007452,-0.021223,0.002276,-0.010338,-0.018511,-0.008365,-0.007359,-0.009605,-0.008877,0.007938,0.049949,-0.009244,-0.023458,0.002910,-0.008446,-0.007764,-0.012257,-0.009714,0.215207,-0.009006,0.129767,0.138836,-0.007785,-0.004129,-0.010275,-0.010475,...,-0.009745,-0.008418,0.057920,-0.011468,-0.007727,-0.006924,-0.009604,-0.008077,-0.009720,-0.010803,0.104902,-0.013248,0.024854,-0.009898,-0.009191,-0.008440,-0.007637,-0.008146,-0.014018,-0.002333,-0.013945,0.045476,-0.007750,-0.009202,-0.010812,-0.008536,-0.007818,-0.009140,-0.008066,-0.007012,-0.008001,-0.009248,-0.012056,-0.005002,-0.010494,-0.010230,-0.005300,-0.007124,-0.011065,-0.007060
+44,-0.009446,1.000000,-0.006157,-0.006541,-0.008925,-0.009749,-0.011799,0.047365,-0.005874,-0.007955,0.029659,-0.009005,-0.008509,0.010565,-0.005779,0.182827,0.029603,-0.008017,-0.014354,0.063629,-0.005707,0.042760,-0.006884,-0.006067,-0.015290,-0.007168,-0.018465,0.017289,-0.006549,0.115444,0.068502,-0.007533,-0.005867,-0.006984,-0.008575,-0.021466,-0.006037,-0.001729,-0.007968,0.053729,...,-0.007557,-0.006528,0.070593,0.003505,0.019900,-0.005369,-0.007447,-0.006263,-0.007538,-0.008378,-0.017528,0.037458,0.284055,-0.007675,-0.007127,-0.006545,-0.005922,-0.006317,-0.010871,0.051438,-0.010814,-0.017943,-0.006009,-0.007135,-0.008385,0.204882,-0.006062,-0.007088,0.185426,-0.005438,-0.006205,-0.007171,-0.009349,-0.009959,-0.008137,-0.007933,-0.009452,-0.005524,-0.008581,-0.005475
...And You Will Know Us by the Trail of Dead,-0.007940,-0.006157,1.000000,-0.005498,-0.007502,-0.008195,-0.009919,0.017846,-0.004938,-0.006687,-0.018554,-0.007569,-0.007153,-0.004793,-0.004858,-0.013834,-0.008446,-0.006739,0.175928,-0.005453,0.002532,-0.006261,-0.005787,-0.005100,-0.012852,-0.006026,0.006328,-0.010882,-0.005505,-0.005061,-0.007989,-0.006332,-0.004931,-0.005871,-0.007208,-0.018044,-0.005075,-0.017535,-0.006698,-0.006828,...,-0.006352,-0.005487,-0.007700,-0.007476,0.061846,-0.004513,-0.006260,-0.005265,-0.006336,-0.007042,-0.014734,-0.008636,0.002433,-0.006452,0.067800,-0.005502,0.015556,-0.005310,-0.009138,0.010474,0.045361,-0.015083,-0.005052,-0.005998,-0.007048,-0.005564,-0.005096,-0.005958,-0.005258,-0.004571,-0.005216,-0.006028,-0.007859,-0.008372,-0.006840,-0.006669,-0.007945,-0.004644,-0.007213,-0.004602
12 Stones,0.000842,-0.006541,-0.005498,1.000000,-0.007969,-0.008706,0.002218,0.075314,-0.005246,0.159073,0.013854,-0.008041,-0.007598,-0.009117,-0.005160,-0.014696,-0.008972,-0.007159,-0.004522,-0.005793,-0.005096,-0.006651,-0.006147,-0.005418,-0.006313,-0.006401,-0.023066,-0.011560,-0.005848,-0.005376,-0.008487,-0.006727,-0.005239,-0.006236,-0.007657,-0.019169,-0.005391,-0.001350,0.160986,-0.007254,...,-0.006748,-0.005829,-0.008180,-0.007941,-0.005351,-0.004795,-0.006650,-0.005593,-0.006731,-0.007481,-0.015652,-0.009174,0.036377,0.142394,-0.006365,-0.005844,-0.005288,-0.005641,-0.009707,-0.011344,-0.009657,-0.003133,-0.005366,-0.00

In [ ]:
def get_similar(artist_name,weight):
    similar_weights = corrMatrix[artist_name]*(weight-2.5)
    similar_weights = similar_weights.sort_values(ascending=False)
    return similar_weights

In [ ]:
artist_sample = [("Duran Duran",18727), ("Duran Duran",13883), ("Duran Duran",13883)]
similar_artist = pd.DataFrame()
for artist,weight in artist_sample:
    similar_artist = similar_artist.append(get_similar(artist,weight),ignore_index = True)

similar_artist.head(5)

,*NSYNC,+44,...And You Will Know Us by the Trail of Dead,12 Stones,2NE1,2Pac,3 Doors Down,30 Seconds to Mars,311,36 Crazyfists,3OH!3,4minute,50 Cent,65daysofstatic,:wumpscut:,A Day to Remember,A Fine Frenzy,A Flock of Seagulls,A Perfect Circle,A Rocket to the Moon,A Silver Mt. Zion,A Skylit Drive,A Tribe Called Quest,A*Teens,ABBA,ABC,AC/DC,AFI,AFX,ASIAN KUNG-FU GENERATION,ATB,Aaliyah,Aaron Carter,Accept,Ace of Base,Adam Lambert,Adam and the Ants,Adele,Adema,Adriana Calcanhotto,...,Yngwie Malmsteen,Yo La Tengo,You Me At Six,ZZ Top,Zebrahead,Zeca Baleiro,Zero 7,Zoé,Zемфира,[unknown],a-ha,blessthefall,blink-182,deadmau5,dredg,happysad,iamamiwhoami,mind.in.a.box,múm,nevershoutnever!,of Montreal,t.A.T.u.,the GazettE,Édith Piaf,Ólafur Arnalds,Агата Кристи,Ария,Дельфин,Кино,Ляпис Трубецкой,Психея,Сплин,倖田來未,宇多田ヒカル,安室奈美恵,植松伸夫,浜崎あゆみ,동방신기,소녀시대,티아라
0,-344.789937,-352.012552,-295.904009,-314.342256,-262.037561,-468.529476,-567.058925,-107.861733,882.185508,-382.305759,-1060.727655,-432.756315,-337.856112,19.411548,1306.659190,-790.928686,-482.87625,4246.211661,-412.943621,-311.745020,-274.260855,-357.962741,-330.835975,-291.580949,1831.764271,5905.741330,352.475522,-622.126380,-314.751889,-289.324147,-217.581963,-362.017600,-281.938337,-335.633008,144.170109,-1031.612162,2094.005644,-1002.496243,-382.917580,-390.367342,...,-363.165668,-313.722165,-440.222598,904.580000,-287.958888,-258.041323,1364.626423,797.368359,1356.929854,285.812227,9094.657265,-493.711731,-1068.117228,-368.862841,-342.531688,-32.515467,-284.598365,-36.547707,-522.419116,-610.488970,-519.697522,-452.481784,-288.803196,1172.857468,-402.948016,27.443790,-291.341885,611.573632,677.031197,238.160453,-298.189198,-159.029943,-449.296159,-438.112932,-391.069936,-381.257846,-420.968095,-265.476781,-412.371883,-263.105761
1,-255.593299,-260.947434,-219.354087,-233.022387,-194.248838,-347.321605,-420.361633,-79.958065,653.965443,-283.403834,-786.319005,-320.802907,-250.453243,14.389810,968.628422,-586.316624,-357.95689,3147.723088,-306.115727,-231.097052,-203.309984,-265.358318,-245.249206,-216.149396,1357.889608,4377.934926,261.290634,-461.183221,-233.326049,-214.476425,-161.293837,-268.364191,-209.001313,-248.805253,106.873518,-764.735647,1552.289532,-743.151972,-283.857378,-289.379897,...,-269.215256,-232.562712,-326.337674,670.566514,-213.464357,-191.286421,1011.599619,591.090363,1005.894141,211.873033,6741.883103,-365.989249,-791.796907,-273.438578,-253.919255,-24.103764,-210.973196,-27.092870,-387.270076,-452.556391,-385.252555,-335.425427,-214.090243,869.441005,-298.705970,20.344123,-215.972177,453.360453,501.884244,176.548702,-221.048101,-117.889136,-333.063918,-324.773775,-289.900732,-282.627015,-312.064281,-196.798337,-305.691896,-195.040696
2,-255.593299,-260.947434,-219.354087,-233.022387,-194.248838,-347.321605,-420.361633,-79.958065,653.965443,-283.403834,-786.319005,-320.802907,-250.453243,14.389810,968.628422,-586.316624,-357.95689,3147.723088,-306.115727,-231.097052,-203.309984,-265.358318,-245.249206,-216.149396,1357.889608,4377.934926,261.290634,-461.183221,-233.326049,-214.476425,-161.293837,-268.364191,-209.001313,-248.805253,106.873518,-764.735647,1552.289532,-743.151972,-283.857378,-289.379897,...,-269.215256,-232.562712,-326.337674,670.566514,-213.464357,-191.286421,1011.599619,591.090363,1005.894141,211.873033,6741.883103,-365.989249,-791.796907,-273.438578,-253.919255,-24.103764,-210.973196,-27.092870,-387.270076,-452.556391,-385.252555,-335.425427,-214.090243,869.441005,-298.705970,20.344123,-215.972177,453.360453,501.884244,176.548702,-221.048101,-117.889136,-333.063918,-324.773775,-289.900732,-282.627015,-312.064281,-196.798337,-305.691896,-195.040696


## Recomender System Using KNN

In [ ]:
#importing the required libraries
import numpy as np
import pandas as pd
import pickle
import scipy.sparse as sp

In [ ]:
weights = pd.read_table('user_artists.dat', encoding="utf-8",sep="\t")
weights.head()

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983


In [ ]:
# Memeriksa apakah pengguna telah menilai film yang sama dua kali, dalam hal ini kami hanya mengambil maksimal dari mereka
weights_df = weights.groupby(['userID','artistID']).aggregate(np.max)

In [ ]:
weights_df.head()

weight
userID artistID        
2      51         13883
       52         11690
       53         11351
       54         10300
       55          8983

In [ ]:
#membuat tabel matriks user item
#gantikan nilai NAN dengan 0
artist_wide = weights.pivot(index = 'artistID', columns = 'userID', values = 'weight').fillna(0)
artist_wide.head()

userID,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,...,2055,2056,2057,2058,2059,2060,2061,2062,2064,2065,2066,2067,2068,2069,2070,2071,2072,2075,2076,2077,2078,2079,2080,2081,2083,2084,2085,2086,2087,2088,2090,2091,2092,2093,2094,2095,2096,2097,2099,2100
artistID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,212.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,83.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,408.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.neighbors import NearestNeighbors

# menentukan nilai parameter model
model_knn = NearestNeighbors(metric='cosine',algorithm='brute')
# fitting model
model_knn.fit(artist_wide)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [ ]:
model_knn.fit(artist_wide)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [ ]:
artist_list = pd.read_table('artists.dat', encoding="utf-8",sep="\t")
artist_list.head()

,id,name,url,pictureURL
0,1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
1,2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
2,3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
3,4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
4,5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...


In [ ]:
#Menentukan 10 tetangga terdekat teratas yang mendapatkan filmnya
def print_similar_artist(query_index) :
    #mengambil peringkat pengguna untuk userId tertentu
    query_index_artist_weights = artist_wide.loc[query_index,:].values.reshape(1,-1)
    #mengambil 10 artist dengan jarak terdekat menampilkan jaraknya
    distances,indices = model_knn.kneighbors(query_index_artist_weights,n_neighbors = 11) 
    #looping untuk menampilkan artist yang mirip 
    for i in range(0,len(distances.flatten())):
        #menampilkan nama artist acak yang dipilih
        get_artist = artist_list.loc[artist_list['id']==query_index]['name']
        #menampilkannama artist pertama dalam daftar, menampilkan artist termirip
        if i==0:
            print('Direkomendasikan {0}:\n'.format(get_artist))
        else :
            #get the indiciees for the closest movies
            indices_flat = indices.flatten()[i]
            #get the title of the movie
            get_artist = artist_list.loc[artist_list['id']==artist_wide.iloc[indices_flat,:].name]['name']
            #print the movie
            print('{0}: {1}, dengan niilai jarak {2}:'.format(i,get_artist,distances.flatten()[i]))

In [ ]:
print_similar_artist(51)

Direkomendasikan 45    Duran Duran
Name: name, dtype: object:

1: 1067    Wham!
Name: name, dtype: object, dengan niilai jarak 0.14449861777037254:
2: 79    Spandau Ballet
Name: name, dtype: object, dengan niilai jarak 0.14571657717437236:
3: 2542    Arcadia
Name: name, dtype: object, dengan niilai jarak 0.17043355334348675:
4: 2536    The Power Station
Name: name, dtype: object, dengan niilai jarak 0.1793333615763517:
5: 8762    Andy Taylor
Name: name, dtype: object, dengan niilai jarak 0.19746357866212239:
6: 12673    Private
Name: name, dtype: object, dengan niilai jarak 0.19797017218461288:
7: 4229    Nephew
Name: name, dtype: object, dengan niilai jarak 0.19831252888421746:
8: 6218    TV-2
Name: name, dtype: object, dengan niilai jarak 0.20756969349503718:
9: 71    George Michael
Name: name, dtype: object, dengan niilai jarak 0.21383856081519625:
10: 987    Mike & The Mechanics
Name: name, dtype: object, dengan niilai jarak 0.2507334845073145:


In [ ]:
distances,indices = model_knn.kneighbors(artist_wide,n_neighbors = 11)

In [ ]:
KNN_SSE=sum(sum(distances))
KNN_SSE

28554.51542743648

## Recomender System using SVM

In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
# menghitung similaritas cosine
cosine_sim = linear_kernel(artist_wide,artist_wide)

In [ ]:
# Similaritas artist based on content
cosine_sim

array([[  284009.,        0.,        0., ...,        0.,        0.,
               0.],
       [       0., 15069270.,        0., ...,        0.,        0.,
               0.],
       [       0.,        0.,   276733., ...,        0.,        0.,
               0.],
       ...,
       [       0.,        0.,        0., ...,    82369.,    82082.,
               0.],
       [       0.,        0.,        0., ...,    82082.,    81796.,
               0.],
       [       0.,        0.,        0., ...,        0.,        0.,
          181476.]])

In [ ]:
#Mengambil 10 artist paling mirip berdasarkan konten
def get_similar_artist_based_on_content_SVM(artist_index) :
    sim_scores = list(enumerate(cosine_sim[artist_index]))
    # Menurutkan artist berdasarkan skor similaritas
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
   
    # Mengambil skor dari 10 artist yang termirip
    sim_scores = sim_scores[0:10]

    artist_indices = [i[0] for i in sim_scores]

    similar_artist = pd.DataFrame(artist_list[['name']].iloc[artist_indices])
    return similar_artist

In [ ]:
get_similar_artist_based_on_content_SVM(51)

,name
45,Duran Duran
47,Air
148,Radiohead
46,Morcheeba
48,Hooverphonic
51,Thievery Corporation
49,Kylie Minogue
50,Daft Punk
59,Coldplay
57,Enigma


In [ ]:
np.shape(cosine_sim)

(17632, 17632)

In [ ]:
weights

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983
...,...,...,...
92829,2100,18726,337
92830,2100,18727,297
92831,2100,18728,281
92832,2100,18729,280


In [ ]:
#membuat tabel matriks user item
#gantikan nilai NAN dengan 0
artist_wide2 = pd.crosstab(weights.artistID, weights.artistID, weights.weight, aggfunc=np.sum)
artist_wide2.head()
np.shape(artist_wide2)

(17632, 17632)

In [ ]:
artist_wide2

artistID,1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,30,31,32,34,36,37,38,39,40,41,43,44,45,...,18696,18697,18698,18706,18707,18708,18709,18710,18711,18712,18713,18714,18715,18716,18717,18718,18719,18720,18721,18722,18723,18724,18725,18726,18727,18728,18729,18730,18734,18735,18736,18737,18738,18739,18740,18741,18742,18743,18744,18745
artistID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,771.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,8012.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,775.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,563.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,913.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,301.0,NaN,NaN,NaN,NaN
18742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,294.0,NaN,NaN,NaN
18743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,287.0,NaN,NaN


In [ ]:
SSE = sum(artist_wide2-cosine_sim)
SSE

161449821

## Caomparation Result DNN , KNN , SVM

|Recomendation System Method | SSE|
-----------------------------|-----
DNN | 7976571
KNN | 28554.51542743648
SVM | 161449821

**#Note:**

1. Untuk pembuatan model DNN sibuatkan 3 fungsi recomenderV1, recomenderV2, dan recomenderNet, hasil terbaik dengan val_los terendah adalah recomenderNet

2. DNN dikomparasikan dengan KNN dan SVM, SSE disajikan di atas

3. Untuk mengatasi data baru, masing2 sudah dibuatkan fungsi untuk merekomendasikan 10 artist didasarkan data baru tersebut

